In [32]:
import pandas as pd

In [33]:
# set pandas so that it shows all rows and all cols
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [34]:
df = pd.read_csv('tweets_labelled_09042020_16072020.csv', sep=';')
df.head()

,id,created_at,text,sentiment
0,77522,2020-04-15 01:03:46+00:00,"RT @RobertBeadles: Yo💥\nEnter to WIN 1,000 Mon...",positive
1,661634,2020-06-25 06:20:06+00:00,#SriLanka surcharge on fuel removed!\n⛽📉\nThe ...,negative
2,413231,2020-06-04 15:41:45+00:00,Net issuance increases to fund fiscal programs...,positive
3,760262,2020-07-03 19:39:35+00:00,RT @bentboolean: How much of Amazon's traffic ...,positive
4,830153,2020-07-09 14:39:14+00:00,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,positive


In [35]:
df.isna().sum()

id               0
created_at       0
text             0
sentiment     3700
dtype: int64

In [36]:
# extract all tickets in the format $TICKER from the 'text' column
# the ticker must consist of letters only (no digits)
df['ticker'] = df['text'].str.findall(r'\$[A-Za-z]+')

# remove the dollar sign and convert tickers to uppercase
df['ticker'] = df['ticker'].apply(lambda x: [t.replace('$', '').upper() for t in x])

# manually explode the list into individual rows by expanding the DataFrame
df_expanded = df.explode('ticker', ignore_index=True)

df_expanded


,id,created_at,text,sentiment,ticker
0,77522,2020-04-15 01:03:46+00:00,"RT @RobertBeadles: Yo💥\nEnter to WIN 1,000 Mon...",positive,NaN
1,661634,2020-06-25 06:20:06+00:00,#SriLanka surcharge on fuel removed!\n⛽📉\nThe ...,negative,NaN
2,413231,2020-06-04 15:41:45+00:00,Net issuance increases to fund fiscal programs...,positive,NaN
3,760262,2020-07-03 19:39:35+00:00,RT @bentboolean: How much of Amazon's traffic ...,positive,NaN
4,830153,2020-07-09 14:39:14+00:00,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,positive,AMD
...,...,...,...,...,...
15562,627230,2020-06-23 14:08:15+00:00,$ITOX working on a contract with a fortune 500...,NaN,ZOM
15563,627230,2020-06-23 14:08:15+00:00,$ITOX working on a contract with a fortune 500...,NaN,DSS
15564,627230,2020-06-23 14:08:15+00:00,$ITOX working on a contract with a fortune 500...,NaN,HGSH
15565,890123,2020-07-14 23:18:34+00:00,"$DIS it could break the 120 pin, then 125&gt; ...",NaN,DIS


In [37]:
df = df_expanded

In [38]:
df['ticker'].value_counts()

ticker
SPX     938
SPY     655
AAPL    641
AMZN    561
FB      479
       ... 
ZEAL      1
MINI      1
ETHE      1
SOME      1
AMED      1
Name: count, Length: 2060, dtype: int64

In [39]:
import yfinance as yf

def is_valid_ticker(ticker):
    # load data for a given ticker
    stock = yf.Ticker(ticker)
    try:
        # try to get info
        info = stock.info
        return info is not None
    except:
        # raise an exception if it fails
        return False

In [40]:
import os

possible_tickers  = df['ticker'].unique()

# convert all tickers to string
possible_tickers = [str(ticker) for ticker in possible_tickers]

# ignore the numbers
possible_tickers = [ticker for ticker in possible_tickers if not ticker.isdigit()]


# commented out because it executes for too long
# I saved the valid tickers into txt file

output = 'valid_tickers.txt'

#execute if there is no output file in the directory
if not os.path.exists(output):
    with open(output, 'w') as file:
        for ticker in possible_tickers:
            if is_valid_ticker(ticker):
                file.write(ticker + '\n')


In [41]:
path = 'valid_tickers.txt'

valid_tickers = []

with open(path, 'r') as file:
    for line in file:
        clean_line = line.strip().replace('[', '').replace(']', '').replace("'", "").replace(',', '')
        
        if clean_line:
            valid_tickers.append(clean_line)

print(valid_tickers)


['nan', 'AMD', 'SPR', 'AMZN', 'TSLA', 'MSFT', 'AAPL', 'BA', 'FB', 'LUV', 'MMM', 'GE', 'AAL', 'UPS', 'TWTR', 'PFE', 'CBSH', 'PEP', 'MA', 'GOOGL', 'GILD', 'SBUX', 'UAL', 'SAVE', 'CRM', 'SPX', 'BTC', 'VIR', 'SPY', 'QQQ', 'IWM', 'IQ', 'GLUU', 'NFLX', 'DIS', 'INTC', 'SNAP', 'AMAT', 'NIO', 'MRK', 'ODP', 'CIO', 'FDX', 'HPP', 'HIW', 'HYG', 'EBAY', 'M', 'T', 'FOREX', 'TS', 'UONE', 'SAM', 'DJI', 'NDX', 'RUT', 'ES', 'DHX', 'V', 'MCD', 'WMT', 'TGT', 'DG', 'TSCO', 'FANG', 'WORK', 'DLPN', 'ABBV', 'TEVA', 'TDOC', 'VIX', 'VXX', 'UVXY', 'SVXY', 'GOOG', 'GSKY', 'GOLD', 'BYND', 'COST', 'NVDA', 'ZM', 'BABA', 'DOW', 'BB', 'KTOV', 'IDEX', 'BIOC', 'XSPA', 'SH', 'SHOP', 'PAYX', 'ASPU', 'SMPL', 'WBA', 'BBBY', 'LEVI', 'SGH', 'MSM', 'HELE', 'AZ', 'SFIX', 'CSCO', 'STX', 'DKS', 'CNI', 'CERS', 'AHH', 'RDFN', 'CMG', 'SPXS', 'SPXL', 'SPXU', 'JPM', 'C', 'FISV', 'WFC', 'JNJ', 'VFC', 'BRK', 'EEM', 'MRNA', 'PENN', 'MARK', 'TMDI', 'GSK', 'PM', 'EIX', 'BIDU', 'XLF', 'CHWY', 'PTON', 'F', 'ADBE', 'CCL', 'MGM', 'HD', 'XLE', '

In [42]:
# having the valid tickers, we can now filter the dataframe to only include the valid tickers
df = df[df['ticker'].isin(valid_tickers)]
df

,id,created_at,text,sentiment,ticker
4,830153,2020-07-09 14:39:14+00:00,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,positive,AMD
6,472959,2020-06-09 05:23:06+00:00,$863.69 Million in Sales Expected for Spirit A...,positive,SPR
10,223041,2020-04-27 00:41:06+00:00,RT @PipsToDollars: Earnings $AMZN $TSLA $MSFT ...,neutral,AMZN
11,223041,2020-04-27 00:41:06+00:00,RT @PipsToDollars: Earnings $AMZN $TSLA $MSFT ...,neutral,TSLA
12,223041,2020-04-27 00:41:06+00:00,RT @PipsToDollars: Earnings $AMZN $TSLA $MSFT ...,neutral,MSFT
...,...,...,...,...,...
15562,627230,2020-06-23 14:08:15+00:00,$ITOX working on a contract with a fortune 500...,NaN,ZOM
15563,627230,2020-06-23 14:08:15+00:00,$ITOX working on a contract with a fortune 500...,NaN,DSS
15564,627230,2020-06-23 14:08:15+00:00,$ITOX working on a contract with a fortune 500...,NaN,HGSH
15565,890123,2020-07-14 23:18:34+00:00,"$DIS it could break the 120 pin, then 125&gt; ...",NaN,DIS


In [43]:
# drop sentiment column
df.drop(columns='sentiment', inplace=True)

C:\Users\48726\AppData\Local\Temp\ipykernel_3432\308094904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='sentiment', inplace=True)


In [44]:
df.head()

,id,created_at,text,ticker
4,830153,2020-07-09 14:39:14+00:00,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,AMD
6,472959,2020-06-09 05:23:06+00:00,$863.69 Million in Sales Expected for Spirit A...,SPR
10,223041,2020-04-27 00:41:06+00:00,RT @PipsToDollars: Earnings $AMZN $TSLA $MSFT ...,AMZN
11,223041,2020-04-27 00:41:06+00:00,RT @PipsToDollars: Earnings $AMZN $TSLA $MSFT ...,TSLA
12,223041,2020-04-27 00:41:06+00:00,RT @PipsToDollars: Earnings $AMZN $TSLA $MSFT ...,MSFT


In [46]:
# after a brief analysis I noticed, that there are some problems with the 'text' column

#the problem is that some text contain ; symbol, so we need to take care of that
df['text'] = df['text'].str.replace(';', ',')
#we also need to clear all new line signs from the text
df['text'] = df['text'].str.replace('\n', ' ')

# I replace the semicolon with a coma and new line with a space, what is not ideal, but will work

C:\Users\48726\AppData\Local\Temp\ipykernel_3432\3701072222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace(';', ',')
C:\Users\48726\AppData\Local\Temp\ipykernel_3432\3701072222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace('\n', ' ')


In [47]:
df.to_csv('tweets_labelled_cleared.csv', sep=';', index=False)